In [1]:
%pip install "torch==2.4.0"
%pip install "transformers==4.45.1" "datasets==3.0.1" "accelerate==0.34.2" "trl==0.11.1" "peft==0.13.0"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 797.3/797.3 MB 90.7 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 1.8 MB/s eta 0:00:0000:0100:06m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 2.0 MB/s eta 0:00:0000:0100:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 2.1 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 2.7 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 4.1 MB/s eta 0:00:0000:0100:05
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 4.9 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 4.0 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 3.8 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 4.3 MB/s eta 0:00:0000:0100:02
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.2/176.2 MB 2.8 MB/s eta 0:00:0000:0100:02
   ━━━━━━━━━━━

In [4]:
!pip install vllm==0.8.2 pandas

INFO: pip is looking at multiple versions of opencv-python-headless to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 293.6/293.6 MB 136.0 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.7/3.7 MB 93.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 766.7/766.7 MB 61.2 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 65.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 70.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 MB 75.8 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 71.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 61.4 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 63.2 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 66.5 MB/s eta 0:00:00a 0:00:01


In [1]:
# 1. 허깅페이스 허브에서 데이터셋 로드
from datasets import load_dataset
import random

dataset = load_dataset("iamjoon/esg-survey-reasoning_datasets", split="train")

# 2. OpenAI format으로 데이터 변환을 위한 함수 
def format_data(sample):
    # OpenAI format으로 변환
    return {
        "messages": [
            {
                "role": "system",
                "content": sample['reasoning_system_prompt'],
            },
            {
                "role": "user",
                "content": sample["user_prompt"],
            },
            {
                "role": "assistant",
                "content": sample["reasoning_assistant"].replace('\n\n답변:', '\n답변:')
            },
        ],
    }

# 3. 전체 데이터에 OpenAI 포맷 전처리 적용
formatted_dataset = dataset.map(format_data)

# 4. 데이터를 7:2 비율로 학습/테스트 분할 (HuggingFace 내장 메서드 사용)
split_dataset = formatted_dataset.train_test_split(test_size=0.3, seed=42)
train_dataset = split_dataset['train']
test_dataset = split_dataset['test']

# 5. 결과 확인
print(f"전체 데이터 수: {len(formatted_dataset)}")
print(f"학습 데이터 수: {len(train_dataset)} ({len(train_dataset)/len(formatted_dataset)*100:.1f}%)")
print(f"테스트 데이터 수: {len(test_dataset)} ({len(test_dataset)/len(formatted_dataset)*100:.1f}%)")

전체 데이터 수: 4599
학습 데이터 수: 3219 (70.0%)
테스트 데이터 수: 1380 (30.0%)


In [2]:
# 데이터셋 저장
test_dataset.save_to_disk("test_dataset")

Saving the dataset (0/1 shards):   0%|          | 0/1380 [00:00<?, ? examples/s]

In [3]:
from transformers import  AutoTokenizer

# 허깅페이스 모델 ID
model_id = "Qwen/Qwen2-7B-Instruct"

# Load model and tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_id)

In [4]:
prompt_lst = []
label_lst = []
for messages in test_dataset["messages"]:
    text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=False)
    input = text.split('<|im_start|>assistant\n')[0] + '<|im_start|>assistant\n'
    label = text.split('<|im_start|>assistant\n')[1].split('<|im_end|>')[0]
    prompt_lst.append(input)
    label_lst.append(label)

In [5]:
print(len(prompt_lst))

1380


In [6]:
print(label_lst[0])

근거: 대웅제약은 회사 내에서 체계적인 리스크 관리 정책을 운영하고 있으며, 주요 위험 요소를 사전에 식별하고 대응함으로써 부정적 영향을 최소화하고 있습니다. 각 리스크 유형에 대한 관리는 해당 부서가 주도하며, 대표이사가 전체 리스크 상황을 파악하고 필요한 조치를 결정합니다. 이를 통해 안정적인 경영을 위한 체계적인 자금 집행과 관리가 이루어지고 있음을 알 수 있습니다 (문서 3).
답변: '5) 매우그렇다'


In [7]:
print(label_lst[10])

근거: 검색 결과에는 동국제강 또는 관련 회사가 반품, 교환 환불, 리콜 처리 등에 관한 구체적인 운영 규정을 언급한 내용이 없습니다. 제공된 문서들은 주로 환경 경영 및 온실가스 감축 노력에 관한 내용으로 이루어져 있으며, 소비자 관련 정책이나 고객 서비스에 관한 정보는 포함되어 있지 않습니다.
답변: 알 수 없음.


In [8]:
print(label_lst[-1])

근거: 검색 결과에 따르면, 대웅제약은 윤리경영과 관련하여 기업 내 다양성과 포용성을 중시하며, 임직원의 권리를 보호하고 법적, 윤리적 규제를 준수하기 위한 다양한 제도와 절차를 운영하고 있습니다 (문서 2). 또한, 대웅제약은 일과 삶의 균형을 이루기 위한 다양한 프로그램을 운영하며, 일하기 좋은 기업으로 인정받은 바 있습니다 (문서 6). 이러한 요소들은 직장 내 기본권을 존중하고 강화하는 데 기여하며, 이는 궁극적으로 직원의 만족도를 높이고 기업의 경영성과에 긍정적인 영향을 미칠 수 있습니다.
답변: '5) 매우 중요하다'


In [9]:
import string
from vllm import LLM, SamplingParams

# LLM
llm = LLM(model="iamjoon/qwen2-8b-esg-survey-model-checkpoint-800")

INFO 08-07 08:08:58 [__init__.py:239] Automatically detected platform cuda.
INFO 08-07 08:09:07 [config.py:585] This model supports multiple tasks: {'reward', 'classify', 'generate', 'score', 'embed'}. Defaulting to 'generate'.
INFO 08-07 08:09:07 [config.py:1697] Chunked prefill is enabled with max_num_batched_tokens=8192.
INFO 08-07 08:09:08 [core.py:54] Initializing a V1 LLM engine (v0.8.2) with config: model='iamjoon/qwen2-8b-esg-survey-model-checkpoint-800', speculative_config=None, tokenizer='iamjoon/qwen2-8b-esg-survey-model-checkpoint-800', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.float16, max_seq_len=32768, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto,  device_config=cuda, decoding_config=DecodingConfig(guided_decoding_bac

Loading safetensors checkpoint shards:   0% Completed | 0/4 [00:00<?, ?it/s]


INFO 08-07 08:09:12 [loader.py:447] Loading weights took 2.12 seconds
INFO 08-07 08:09:13 [gpu_model_runner.py:1186] Model loading took 14.2487 GB and 2.631303 seconds
INFO 08-07 08:09:19 [backends.py:415] Using cache directory: /root/.cache/vllm/torch_compile_cache/777ce78b85/rank_0_0 for vLLM's torch.compile
INFO 08-07 08:09:19 [backends.py:425] Dynamo bytecode transform time: 6.23 s
INFO 08-07 08:09:19 [backends.py:115] Directly load the compiled graph for shape None from the cache
INFO 08-07 08:09:28 [monitor.py:33] torch.compile takes 6.23 s in total
INFO 08-07 08:09:28 [kv_cache_utils.py:566] GPU KV cache size: 953,616 tokens
INFO 08-07 08:09:28 [kv_cache_utils.py:569] Maximum concurrency for 32,768 tokens per request: 29.10x
INFO 08-07 08:09:47 [gpu_model_runner.py:1534] Graph capturing finished in 19 secs, took 0.48 GiB
INFO 08-07 08:09:48 [core.py:151] init engine (profile, create kv cache, warmup model) took 35.00 seconds


In [10]:
len(prompt_lst)

1380

In [11]:
# 긴 프롬프트 샘플 배제
def filter_long_prompts(prompt_lst, label_lst, max_length=8000):
    """
    최대 길이를 초과하는 프롬프트 샘플들을 배제하는 함수
    """
    filtered_prompts = []
    filtered_labels = []
    filtered_indices = []
    
    for i, prompt in enumerate(prompt_lst):
        if len(prompt) <= max_length:
            filtered_prompts.append(prompt)
            filtered_labels.append(label_lst[i])
            filtered_indices.append(i)
    
    print(f"전체 {len(prompt_lst)}개 중 {len(filtered_prompts)}개 샘플 유지")
    print(f"{len(prompt_lst) - len(filtered_prompts)}개 샘플 배제됨")
    
    return filtered_prompts, filtered_labels, filtered_indices

# 사용법
filtered_prompt_lst, filtered_label_lst, valid_indices = filter_long_prompts(prompt_lst, label_lst, max_length=13000)
print(len(filtered_prompt_lst))
print(len(filtered_label_lst))

전체 1380개 중 216개 샘플 유지
1164개 샘플 배제됨
216
216


In [12]:
# 샘플링 파라미터 설정
sampling_params = SamplingParams(
    temperature=0,
    max_tokens=1024,
    stop=["\n<|end_of_text|>", "<|eot_id|>"],  # 문자열로 중지 토큰 지정
)

# 생성
outputs = llm.generate(filtered_prompt_lst, sampling_params)

Processed prompts: 100%|██████████| 216/216 [02:20<00:00,  1.54it/s, est. speed input: 12666.28 toks/s, output: 201.09 toks/s]


In [13]:
# outputs를 문자열 리스트로 파싱
preds = [output.outputs[0].text for output in outputs]

In [14]:
preds[:5]

["근거: 검색 결과에 따르면, 대상㈜는 개인정보의 보유 기간을 명시하고 있으며, 일반 개인정보는 회원 탈퇴 시까지 보관하고, 일부 개인정보는 법령에 정한 기간 동안 보관한다고 명시하고 있습니다 (문서 7). 따라서 소비자의 정보를 수집하는 과정에서 수집된 정보의 유효기간을 정확히 명시하고 있음을 알 수 있습니다.\n답변: '1) 예'",
 "근거: 삼양홀딩스의 2021 지속가능경영보고서에 따르면, 육아휴직 후 복귀한 수는 2021년에 2명이었으며, 육아휴직 사용 후 업무 복귀율은 100%였습니다 (문서 0). 이는 여성 직원의 육아휴직 이후 업무 복귀 및 근속률이 매우 높다는 것을 의미합니다.\n답변: '5) 매우 많다'",
 '근거: 검색 결과에 따르면, 동국제강은 환경경영과 관련된 다양한 활동을 수행하고 있으며, 온실가스 감축 및 에너지 효율 향상을 위한 노력을 기울이고 있습니다. 그러나 반품, 교환, 환불, 리콜 처리 등에 대한 구체적인 운영 규정에 대한 정보는 제공되지 않았습니다.\n답변: 알 수 없음.',
 "근거: 검색 결과에는 '고용 및 근로조건'에 대한 직접적인 언급이 없습니다. 따라서 '고용 및 근로조건'이 기업의 경영성과에 미치는 중요성을 평가할 수 있는 정보가 없습니다.\n답변: 알 수 없음.",
 '근거: 검색 결과에서는 동국제강의 환경경영 및 저탄소 사회 실현에 대한 다양한 노력을 기술하고 있지만, 업무 현장의 산재 사고 예방을 위한 위험감지센터 설치에 대한 구체적인 언급은 없습니다.\n답변: 알 수 없음.']

In [15]:
filtered_label_lst[:5]

["근거: 대상㈜는 개인정보처리방침을 통해 이해관계자들이 자신의 개인정보를 어떻게 처리하고 있는지를 쉽고 명확하게 안내하고 있으며, 개인정보의 보유 기간을 명시하고 있습니다. 대부분의 개인정보는 회원 탈퇴 시까지 보관되며, 일부 개인정보는 법령에 정한 기간 동안 보관된다고 설명하고 있습니다 (문서 7).\n답변: '1) 예'",
 "근거: 삼양홀딩스의 2021 지속가능경영보고서를 보면 2021년 여성 육아휴직 사용자 수는 8명이었고, 이 중 2명이 육아휴직 후 복귀했습니다. 그러나, 육아휴직을 사용한 여성 직원의 복귀율은 100%로 나타나고 있으며, 이는 휴직을 쓴 직원들이 다시 업무에 복귀하고 있다는 것을 의미합니다 (문서 0).\n답변: '4) 많다'",
 '근거: 검색 결과에는 동국제강 또는 관련 회사가 반품, 교환 환불, 리콜 처리 등에 관한 구체적인 운영 규정을 언급한 내용이 없습니다. 제공된 문서들은 주로 환경 경영 및 온실가스 감축 노력에 관한 내용으로 이루어져 있으며, 소비자 관련 정책이나 고객 서비스에 관한 정보는 포함되어 있지 않습니다.\n답변: 알 수 없음.',
 "근거: 검색 결과에서는 주어진 질문과 관련된 직접적인 정보는 없으나, 동국제강의 환경 경영 및 저탄소 사회 실현을 위한 노력이 언급되어 있습니다. 일반적으로 기업의 고용 및 근로조건은 직원 만족도와 생산성에 영향을 미치며, 이는 다시 기업의 경영성과에 중요한 역할을 합니다. 따라서, 이러한 요인들이 직결되지 않더라도 일반적인 경영 관점에서는 중대한 역할을 할 수 있습니다.\n답변: '5) 매우 중요하다'",
 '근거: 검색 결과에서 동국제강의 경우, 환경 안전 팀이 전반적인 활동의 실행 주체로 환경 이슈를 신속하게 대응하고 있으며, 주기적인 전사 회의를 통해 조직적인 관리가 진행되고 있다는 내용이 있습니다 (문서 5). 하지만, 산재 사고 예방을 위한 위험감지센터 설치 여부에 관한 구체적인 정보는 확인되지 않았습니다.\n답변: 알 수 없음.']

In [16]:
import re

def calculate_accuracy(labels_list, predictions_list):
    """
    텍스트에서 '답변:' 다음 부분을 추출하여 일치도를 계산하는 함수
    """
    def extract_answer(text):
        match = re.search(r'답변:\s*(.+)', text.strip())
        if match:
            return match.group(1).strip()
        return ""
    
    if len(labels_list) != len(predictions_list):
        raise ValueError("라벨과 예측 결과의 개수가 일치하지 않습니다.")
    
    correct_count = 0
    total_count = len(labels_list)
    
    for label_text, pred_text in zip(labels_list, predictions_list):
        label_answer = extract_answer(label_text)
        pred_answer = extract_answer(pred_text)
        
        if label_answer == pred_answer:
            correct_count += 1
    
    accuracy = correct_count / total_count if total_count > 0 else 0.0
    
    return accuracy

In [17]:
# checkpoint-800
calculate_accuracy(filtered_label_lst, preds)

0.6435185185185185

In [17]:
# checkpoint-1600
calculate_accuracy(filtered_label_lst, preds)

0.6342592592592593

In [17]:
# checkpoint-2412
calculate_accuracy(filtered_label_lst, preds)

0.5